# Exercise 05: Secrets and Security in GitHub Actions

In this notebook, we'll explore how to securely handle sensitive information in GitHub Actions using secrets and security best practices.

## Prerequisites

Before starting this exercise, ensure you have:

1. **AWS Account Setup**
   - An AWS account with appropriate permissions
   - An S3 bucket for deployment
   - An IAM role with necessary permissions
   - AWS credentials configured

2. **GitHub Repository**
   - A repository with GitHub Actions enabled
   - Appropriate permissions to manage secrets
   - Access to create environments

3. **Security Tools**
   - OWASP Dependency Check
   - Gitleaks for secret scanning
   - Java 17 or later (for OWASP Dependency Check)

## Understanding GitHub Secrets

GitHub Secrets are encrypted environment variables that you can use to store sensitive information like:

- API keys
- Passwords
- Tokens
- Certificates

There are three types of secrets in GitHub:

1. Repository secrets
2. Organization secrets
3. Environment secrets

## Basic Secrets Usage

Here's how to use secrets in a workflow:

```yaml
name: Basic Secrets Example

on: [push]

jobs:
  secure-deploy:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Use repository secret
        run: |
          # Never print secrets directly!
          echo "Using API key: ${{ secrets.API_KEY }}"
      
      - name: Use environment secret
        env:
          DB_PASSWORD: ${{ secrets.DB_PASSWORD }}
        run: |
          # Use the secret securely
          ./deploy.sh
```

## Advanced Security Features

GitHub Actions provides several advanced security features:

1. **Environment Protection Rules**
   - Required reviewers
   - Wait timer
   - Branch restrictions

2. **OIDC Integration**
   - Cloud provider authentication
   - Short-lived credentials
   - No stored secrets

3. **Permission Settings**
   - Workflow permissions
   - Token permissions
   - Environment access

Example of advanced security configuration:

```yaml
name: Advanced Security Example

on: [push]

# Define workflow-level permissions
permissions:
  contents: read
  packages: write
  security-events: write
  
jobs:
  secure-workflow:
    runs-on: ubuntu-latest
    environment:
      name: production
      url: https://prod.example.com
    
    steps:
      - uses: actions/checkout@v4
      
      - name: Configure AWS Credentials
        uses: aws-actions/configure-aws-credentials@v4
        with:
          role-to-assume: ${{ secrets.AWS_ROLE_ARN }}
          aws-region: us-east-1
```

## Security Scanning Tools

GitHub Actions can integrate with various security scanning tools to enhance your security posture:

### 1. OWASP Dependency Check

OWASP Dependency Check is a tool that identifies project dependencies and checks if there are any known, publicly disclosed, vulnerabilities.

```yaml
jobs:
  security-checks:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
        with:
          # Limit checkout depth for security
          fetch-depth: 1
      
      - name: Set up Java
        uses: actions/setup-java@v4
        with:
          distribution: 'temurin'
          java-version: '17'
      
      - name: Run OWASP Dependency Check
        uses: dependency-check/Dependency-Check_Action@main
        with:
          project: 'MyProject'
          path: '.'
          format: 'HTML'
          out: 'reports'
          args: >
            --suppression suppression.xml
            --failOnCVSS 7
            --enableRetired
```

#### Handling False Positives

You can create a `suppression.xml` file to handle false positives:

```xml
<?xml version="1.0" encoding="UTF-8"?>
<suppressions xmlns="https://jeremylong.github.io/DependencyCheck/dependency-suppression.1.3.xsd">
    <suppress>
        <notes>False positive for example package</notes>
        <packageUrl regex="true">pkg:npm/example@.*</packageUrl>
        <cve>CVE-2023-XXXX</cve>
    </suppress>
</suppressions>
```

### 2. Gitleaks

Gitleaks is a tool for detecting hardcoded secrets like passwords, API keys, and tokens in your codebase.

```yaml
jobs:
  security-checks:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Check for exposed secrets
        uses: gitleaks/gitleaks-action@v2
```

### 3. CodeQL Analysis

CodeQL is GitHub's semantic code analysis engine that helps identify security vulnerabilities in your code.

```yaml
jobs:
  security-checks:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Initialize CodeQL
        uses: github/codeql-action/init@v2
        with:
          languages: javascript
      
      - name: Perform CodeQL Analysis
        uses: github/codeql-action/analyze@v2
```

## Secret Rotation and Management

Best practices for managing secrets:

1. **Regular Rotation**
   - Automate secret rotation
   - Use cloud provider features
   - Implement rotation workflows

2. **Access Control**
   - Limit secret access
   - Use environment protection
   - Audit secret usage

Example of a secret rotation workflow using AWS Secrets Manager:

```yaml
name: Secret Rotation

on:
  workflow_dispatch:
    inputs:
      environment:
        description: "Environment to rotate secrets for"
        required: true
        default: "production"
        type: choice
        options:
          - staging
          - production

jobs:
  rotate-secrets:
    runs-on: ubuntu-latest
    environment: ${{ github.event.inputs.environment }}
    
    steps:
      - uses: actions/checkout@v4
      
      - name: Configure AWS Credentials
        uses: aws-actions/configure-aws-credentials@v4
        with:
          role-to-assume: ${{ secrets.AWS_ROLE_ARN }}
          aws-region: ${{ secrets.AWS_REGION }}
      
      - name: Generate new API key
        id: generate-key
        run: |
          NEW_KEY=$(openssl rand -hex 32)
          echo "new_key=$NEW_KEY" >> $GITHUB_OUTPUT
      
      - name: Update API key in AWS Secrets Manager
        run: |
          aws secretsmanager update-secret --secret-id ${{ secrets.AWS_SECRET_ID }} \
            --secret-string "{\"API_KEY\":\"${{ steps.generate-key.outputs.new_key }}\"}"
```

## Security Best Practices

When working with secrets in GitHub Actions:

1. **Never log secrets**: Avoid printing or logging secret values
2. **Use minimal permissions**: Set least privilege access
3. **Rotate secrets regularly**: Implement secret rotation
4. **Use environment protection**: Add required reviewers
5. **Audit secret usage**: Monitor who accesses secrets
6. **Use OIDC when possible**: For cloud provider authentication
7. **Validate secret values**: Check format and validity
8. **Use secret scanning**: Enable GitHub's secret scanning
9. **Run security checks**: Use OWASP Dependency Check and Gitleaks
10. **Handle false positives**: Create suppression files for known issues

## Hands-on Exercise

Let's practice what we've learned! Follow these steps:

1. **Set Up AWS Resources**
   - Create an S3 bucket for deployment
   - Set up an IAM role with appropriate permissions
   - Configure AWS credentials in GitHub Secrets

2. **Configure GitHub Secrets**
   - Go to your repository's Settings > Secrets and variables > Actions
   - Add AWS-related secrets:
     - `AWS_ROLE_ARN`
     - `AWS_REGION`
     - `AWS_BUCKET`
     - `AWS_SECRET_ID`
   - Create environments (staging, production)
   - Add environment-specific secrets

3. **Set Up Security Tools**
   - Create a `suppression.xml` file for OWASP Dependency Check
   - Configure Gitleaks for secret scanning
   - Set up CodeQL analysis

4. **Explore the Security Workflow**
   - Look at `.github/workflows/05-secrets-security.yml`
   - Notice the security checks and scanning
   - See how it handles different environments
   - Understand the OIDC authentication with AWS

5. **Run the Workflow**
   - Go to your repository's Actions tab
   - Find the 'Secrets and Security Example' workflow
   - Click 'Run workflow'
   - Select different environments
   - Watch the security checks run

6. **Security Challenges**
   - Add a new security scanning tool
   - Implement secret rotation for AWS credentials
   - Add environment protection rules
   - Create a security audit report

Remember to never commit secrets to your repository and always use GitHub's secret management!